In [3]:
using CSV, DataFrames, Dates, TimeZones

ENV["COLUMNS"] = 1000

1000

In [4]:
ID = "Charger06"

"Charger06"

In [5]:
# Read the CSV file from the specified path
df = CSV.read("data/charger_battery_data_$ID.csv", DataFrame)

# Filter the data
df = df[:, [:timestamp, :e_consumption, :e_production, :e_charger, :h_countdown, :soc_ev]]

df.e_consumption = df.e_consumption / 1000
df.e_production = df.e_production / 1000
df.e_charger = df.e_charger / 1000

# Parse the timestamps with timezone offset
timestamps = ZonedDateTime.(df.timestamp, DateFormat("yyyy-mm-dd HH:MM:SSzzzz"))

# Shift the timestamps one hour ahead
timestamps = timestamps .- Hour(1)

# Replace the timestamp column in the DataFrame with the shifted timestamps
df.timestamp = timestamps

35040-element Vector{ZonedDateTime}:
 2020-11-01T00:00:00+01:00
 2020-11-01T00:15:00+01:00
 2020-11-01T00:30:00+01:00
 2020-11-01T00:45:00+01:00
 2020-11-01T01:00:00+01:00
 2020-11-01T01:15:00+01:00
 2020-11-01T01:30:00+01:00
 2020-11-01T01:45:00+01:00
 2020-11-01T02:00:00+01:00
 2020-11-01T02:15:00+01:00
 ⋮
 2021-10-31T21:45:00+01:00
 2021-10-31T22:00:00+01:00
 2021-10-31T22:15:00+01:00
 2021-10-31T22:30:00+01:00
 2021-10-31T22:45:00+01:00
 2021-10-31T23:00:00+01:00
 2021-10-31T23:15:00+01:00
 2021-10-31T23:30:00+01:00
 2021-10-31T23:45:00+01:00

In [6]:
df

,timestamp,e_consumption,e_production,e_charger,h_countdown,soc_ev
,ZonedDa…,Float64,Float64,Float64?,Float64,Float64
1,2020-11-01T00:00:00+01:00,0.542,0.0,missing,-1.0,1.0
2,2020-11-01T00:15:00+01:00,0.539,0.0,missing,-1.0,1.0
3,2020-11-01T00:30:00+01:00,0.53,0.0,missing,-1.0,1.0
4,2020-11-01T00:45:00+01:00,0.517,0.0,missing,-1.0,1.0
5,2020-11-01T01:00:00+01:00,0.064,0.0,missing,-1.0,1.0
6,2020-11-01T01:15:00+01:00,0.064,0.0,missing,-1.0,1.0
7,2020-11-01T01:30:00+01:00,0.052,0.0,missing,-1.0,1.0
8,2020-11-01T01:45:00+01:00,0.06,0.0,missing,-1.0,1.0
9,2020-11-01T02:00:00+01:00,0.073,0.0,missing,-1.0,1.0


In [7]:
for col_name in names(df)
    display(("Column: ", col_name, ", Type: ", eltype(df[!, col_name])))
end

for col_name in names(df)
    missing_count = sum(ismissing.(df[!, col_name]))
    display(("Column: ", col_name, ", Missing Values: ", missing_count))
end

("Column: ", "timestamp", ", Type: ", ZonedDateTime)

("Column: ", "e_consumption", ", Type: ", Float64)

("Column: ", "e_production", ", Type: ", Float64)

("Column: ", "e_charger", ", Type: ", Union{Missing, Float64})

("Column: ", "h_countdown", ", Type: ", Float64)

...

("Column: ", "e_consumption", ", Missing Values: ", 0)

("Column: ", "e_production", ", Missing Values: ", 0)

("Column: ", "e_charger", ", Missing Values: ", 28546)

("Column: ", "h_countdown", ", Missing Values: ", 0)

("Column: ", "soc_ev", ", Missing Values: ", 0)

In [8]:
#describe(df)

Resamplign from 15min intervalls to 1h intervalls. In the future, I'd like to try to work with 15min intervals.

In [9]:
function resample(df, time_column, interval)
    # Round the timestamps to the nearest hour
    df[!, time_column] = Dates.floor.(df[!, time_column], interval)
    
    # Define the columns to be summed
    sum_columns = ["e_consumption", "e_production", "e_charger"]
    
    # Group by the rounded timestamps and sum the other columns
    new_df = combine(groupby(df, time_column), sum_columns .=> (x -> sum(coalesce.(x, 0))) .=> sum_columns)
    
    # Handle "h_countdown" and "soc_ev" separately
    min_values = combine(groupby(df, time_column), "h_countdown" => minimum => "h_countdown")
    new_df = leftjoin(new_df, min_values, on=time_column)
    
    min_values = combine(groupby(df, time_column), "soc_ev" => minimum => "soc_ev")
    new_df = leftjoin(new_df, min_values, on=time_column)

    #Increase every value of h_countdown to the next higher integer
    new_df.h_countdown = ceil.(new_df.h_countdown)

    # Change last countdown value from -1
    for i in 2:(nrow(new_df) )# - 1)
        if new_df[i, :h_countdown] == -1 && new_df[(i-1), :h_countdown] == 1
            new_df[i, :h_countdown] = 0
        end
    end

    return new_df
end


resample (generic function with 1 method)

In [10]:
# Resample the data to 1-hour intervals
df_resampled = resample(df, :timestamp, Dates.Hour(1))

,timestamp,e_consumption,e_production,e_charger,h_countdown,soc_ev
,ZonedDa…,Float64,Float64,Float64,Float64,Float64?
1,2020-11-01T00:00:00+01:00,2.128,0.0,0.0,-1.0,1.0
2,2020-11-01T01:00:00+01:00,0.24,0.0,0.0,-1.0,1.0
3,2020-11-01T02:00:00+01:00,0.722,0.0,0.0,-1.0,1.0
4,2020-11-01T03:00:00+01:00,2.186,0.0,0.0,-1.0,1.0
5,2020-11-01T04:00:00+01:00,2.162,0.0,0.0,-1.0,1.0
6,2020-11-01T05:00:00+01:00,1.701,0.0,0.0,-1.0,1.0
7,2020-11-01T06:00:00+01:00,0.258,0.147,0.0,-1.0,1.0
8,2020-11-01T07:00:00+01:00,0.764,0.5,0.0,-1.0,1.0
9,2020-11-01T08:00:00+01:00,2.059,1.161,0.0,-1.0,1.0


In [11]:
# Change the type of the chargekwh column to Union{Missing, Float64}
df_resampled.e_charge = convert(Vector{Union{Missing, Float64}}, df_resampled.e_charger)


8760-element Vector{Union{Missing, Float64}}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [12]:
# Create a new DataFrame with the desired structure
new_df = DataFrame(
    electkwh = df_resampled.e_consumption,
    PV_generation = df_resampled.e_production,
    chargekwh = df_resampled.e_charger,
    h_countdown = df_resampled.h_countdown,
    soc_ev = df_resampled.soc_ev,
    month = month.(DateTime.(df_resampled.timestamp)),
    day = day.(DateTime.(df_resampled.timestamp)),
    hour = hour.(DateTime.(df_resampled.timestamp))
)

,electkwh,PV_generation,chargekwh,h_countdown,soc_ev,month,day,hour
,Float64,Float64,Float64,Float64,Float64?,Int64,Int64,Int64
1,2.128,0.0,0.0,-1.0,1.0,11,1,0
2,0.24,0.0,0.0,-1.0,1.0,11,1,1
3,0.722,0.0,0.0,-1.0,1.0,11,1,2
4,2.186,0.0,0.0,-1.0,1.0,11,1,3
5,2.162,0.0,0.0,-1.0,1.0,11,1,4
6,1.701,0.0,0.0,-1.0,1.0,11,1,5
7,0.258,0.147,0.0,-1.0,1.0,11,1,6
8,0.764,0.5,0.0,-1.0,1.0,11,1,7
9,2.059,1.161,0.0,-1.0,1.0,11,1,8


In [13]:

# Change the type of the chargekwh column to Union{Missing, Float64}
new_df.chargekwh = convert(Vector{Union{Missing, Float64}}, new_df.chargekwh)

# In every row where h_countdown is -1, set chargekwh to 'missing'
new_df[new_df.h_countdown .== -1, :chargekwh] .= missing

7129-element view(::Vector{Union{Missing, Float64}}, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  8751, 8752, 8753, 8754, 8755, 8756, 8757, 8758, 8759, 8760]) with eltype Union{Missing, Float64}:
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 ⋮
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing

In [14]:
# Write the new DataFrame to a CSV file in the specified path
CSV.write("data/sonnen$(ID)_datafile_all.csv", new_df)

"data/sonnenCharger06_datafile_all.csv"

In [15]:
#Input_df = CSV.read("data/sonnen$(ID)_datafile_all.csv", DataFrame);

Input_df = new_df

,electkwh,PV_generation,chargekwh,h_countdown,soc_ev,month,day,hour
,Float64,Float64,Float64?,Float64,Float64?,Int64,Int64,Int64
1,2.128,0.0,missing,-1.0,1.0,11,1,0
2,0.24,0.0,missing,-1.0,1.0,11,1,1
3,0.722,0.0,missing,-1.0,1.0,11,1,2
4,2.186,0.0,missing,-1.0,1.0,11,1,3
5,2.162,0.0,missing,-1.0,1.0,11,1,4
6,1.701,0.0,missing,-1.0,1.0,11,1,5
7,0.258,0.147,missing,-1.0,1.0,11,1,6
8,0.764,0.5,missing,-1.0,1.0,11,1,7
9,2.059,1.161,missing,-1.0,1.0,11,1,8


In [16]:
#describe(Input_df)

In [17]:
# add new column with number of days
Input_df[!, :nday] = 1:nrow(Input_df)

1:8760

In [18]:
# calculate residual demand
Input_df[!, :d_res] = Input_df[!,:electkwh] + coalesce.(Input_df[!,:chargekwh], 0) - Input_df[!,:PV_generation]

8760-element Vector{Float64}:
  2.128
  0.24
  0.722
  2.186
  2.162
  1.701
  0.11100000000000002
  0.264
  0.8980000000000001
 -1.584
  ⋮
 -0.617
  0.7210000000000001
  1.69
  0.5630000000000001
  2.612
  2.2880000000000003
  0.9759999999999999
  0.677
  2.005

## Add periodical time representation using cos/sin

In [19]:
# add columns with cos and sin values for periodical time values day + month
Input_df[!, :hour_cos] = cos.(Input_df[!,:hour] ./ maximum(Input_df[!,:hour]) .* 2*pi);
Input_df[!, :hour_sin] = sin.(Input_df[!,:hour] ./ maximum(Input_df[!,:hour]) .* 2*pi);

Input_df[!, :month_cos] = cos.(Input_df[!,:month] ./ maximum(Input_df[!,:month]) .* 2*pi);
Input_df[!, :month_sin] = sin.(Input_df[!,:month] ./ maximum(Input_df[!,:month]) .* 2*pi);

#Input_df[!, :nday_cos] = cos.(Input_df[!,:nday] ./ maximum(Input_df[!,:nday]) .* 2*pi);
#Input_df[!, :nday_sin] = sin.(Input_df[!,:nday] ./ maximum(Input_df[!,:nday]) .* 2*pi);

## Add seasons

In [20]:
Input_df[!, :spring] = (Input_df[!,:month] .>= 3) .* (Input_df[!,:month] .<= 5);
Input_df[!, :summer] = (Input_df[!,:month] .>= 6) .* (Input_df[!,:month] .<= 8);
Input_df[!, :autumn] = (Input_df[!,:month] .>= 9) .* (Input_df[!,:month] .<= 11);
Input_df[!, :winter] = convert.(Bool, (Input_df[!,:month] .>= 12) .+ (Input_df[!,:month] .<= 2));

Input_df[!, :season] = ifelse.(Input_df[!,:spring] .== true, 1,
                        ifelse.(Input_df[!,:summer] .== true, 2,
                        ifelse.(Input_df[!,:autumn] .== true, 3, 
                        4)));

In [21]:
print("L")

L

In [22]:
#describe(Input_df)

## Add dynamic prices based on Ye et al. 2020

In [23]:
#LU
#=
function set_dynamic_prices(Input_df)
    map(eachrow(Input_df)) do r
        if r.month >= 5 && r.month <= 10
            if (r.hour >= 6 && r.hour <= 9) || (r.hour >= 16 && r.hour <= 17)
                return 0.3f0
            elseif (r.hour >= 10 && r.hour <= 15)
                return 0.6f0
            else
                return 0.15f0
            end
        elseif r.month >= 11 || r.month <= 4
            if (r.hour >= 6 && r.hour <= 9) || (r.hour >= 16 && r.hour <= 17)
                return 0.6f0
            elseif r.hour >= 10 && r.hour <= 15
                return 0.3f0
            else
                return 0.15f0
            end
        end
    end
end      =#  

In [24]:
#LU
#=Input_df[!, "p_buy"] = set_dynamic_prices(Input_df);
Input_df[!, "p_sell"] = 0.5 .* Input_df[!, "p_buy"];
=#

In [25]:
#LU describe(Input_df)

## Extract training, testing + evalution data set for summer, winter, both, all

In [26]:
#LU
#=# filter summer
Input_data_summer = filter(:summer => !=(0), Input_df)
describe(Input_data_summer), size(Input_data_summer)
=#

In [27]:
#=function train_eval_test_split(Input_df)
    train = filter(row -> row.day <= 15, Input_df)
    eval = filter(row -> row.day > 15 && row.day <= 20, Input_df)
    test = filter(row -> row.day > 20, Input_df)
    return train, eval, test
end=#

In [28]:
#=function split_all_data_advanced(Input_df)
    # Initialize the split points and the adjustments
    splitpoint_adjustments = Dict("train" => 0, "eval" => 0, "test" => 0)
    splitpoints = Dict("train" => 15, "eval" => 20)#, "test" => lastday(Input_df.date[1]))

    # Initialize the data sets
    train, eval, test = DataFrame(), DataFrame(), DataFrame()

    # Iterate over each month
    for month in unique(Input_df[:,:month])
        # Adjust the split points based on the previous month's adjustments
        #print("\n New month: ", month, "\n Splitpoint adjustment: ", splitpoint_adjustments)
        splitpoints = Dict("train" => 15, "eval" => 20)#, "test" => lastday(Input_df.date[1]))
        splitpoints["train"] -= splitpoint_adjustments["train"]
        splitpoints["eval"] -= splitpoint_adjustments["eval"]
        #splitpoints["test"] -= splitpoint_adjustments["test"]

        #print("\nSplitpoints: ", splitpoints)

        # Reset adjustments for the current month
        splitpoint_adjustments = Dict("train" => 0, "eval" => 0)#, "test" => 0)

        # Filter the data for the current month
        month_data = filter(row -> row.month == month, Input_df)

        # Check and adjust the split points
        for (yset_name, yday) in splitpoints
            #print("\nmonth: ", month, "\nyset_name: ", yset_name, "\n yday: ", yday, "\n")
            while ((month_data[(month_data.day .== yday) .& (month_data.hour .== 23), :h_countdown][1] > -1) && splitpoint_adjustments[yset_name] <= 4)
                print("split not ok at month ", month, ", day ", yday, ". Trying at next day.", "\n")
                yday += 1
                splitpoint_adjustments[yset_name] += 1
                if splitpoint_adjustments[yset_name] == 4
                    print("\n Maximum adjustment of 4 days reached! Not ideal splitpoint chosen at month ", month, ". \n Splitpoint: ", yset_name, " at ", yday)
                end

            end
            
            splitpoints[yset_name] = yday
        end

        # Split the data for the current month
        train = vcat(train, filter(row -> row.day <= splitpoints["train"], month_data))
        eval = vcat(eval, filter(row -> row.day > splitpoints["train"] && row.day <= splitpoints["eval"], month_data))
        test = vcat(test, filter(row -> row.day > splitpoints["eval"], month_data))
    end

    return train, eval, test
end
=#

In [55]:
function split_all_data_advanced_v2(Input_df)
    print("start")
    # Initialize the data sets
    train = DataFrame()
    eval = DataFrame()
    test = DataFrame()

    # Define the pattern of row counts for each data set
    pattern = [("test", 24*10), ("eval", 5*24), ("train", 15*24)]

    splitpoint_adjustments = Dict("train" => 0, "eval" => 0, "test" => 0)

    # Initialize the pattern index
    pattern_index = 1

    i = 1

    # Iterate over the rows of the DataFrame
    while i <= size(Input_df, 1)
        # Get the current data set name and row count from the pattern
        set_name, row_count = pattern[pattern_index]

        row_count -= min(splitpoint_adjustments[set_name], 4*24)

        #print("\n i: $i, set_name: $set_name, row_count: $row_count, days: $(row_count/24)")

        splitpoint_adjustments[set_name] -= min(splitpoint_adjustments[set_name], 4*24)

        print("\n i $i, set_name $set_name, days $(row_count/24), adjustment left: $(splitpoint_adjustments[set_name])")

        while (Input_df[min(i+row_count-1, size(Input_df, 1)), :h_countdown] > -1) #&& splitpoint_adjustments[set_name] <= 3*24)
            print("\n i: $i, set_name: $set_name, row_count: $row_count, days: $(row_count/24)")
            print("\n split point not ok at nday ", Input_df[min(i+row_count-1, size(Input_df, 1)), :nday] ,
                    ".\n month $(Input_df[min(i+row_count-1, size(Input_df, 1)), :month]), day $(Input_df[min(i+row_count-1, size(Input_df, 1)), :day]). Trying at next day. \n")
            row_count += 24
            splitpoint_adjustments[set_name] += 24
            print("Total new splitpoint adjustment for $set_name: $(splitpoint_adjustments[set_name])")
        end

        if splitpoint_adjustments[set_name] == 4*24
            print("\n Maximum splitpoint adjustment of 4 days reached! Non-ideal splitpoint chosen at nday ", Input_df[min(i+row_count-1, size(Input_df, 1)), :nday] , " for set ", set_name)
        end

        print("\n Splitpoint found.")

        # Get the rows for the current data set
        

        # Add the rows to the appropriate data set
        if set_name == "train"
            rows = Input_df[i:min(i+row_count-1, size(Input_df, 1)), :]
            train = vcat(train, rows)

            if nrow(train) # TBC: IF train length > 1440 then clip and aso reduce row_count. At the end turn last value of each set c_ev=0.

        elseif set_name == "eval"
            eval = vcat(eval, rows)
        else # set_name == "test"
            test = vcat(test, rows)
        end

        # Move to the next rows
        i = i + row_count

        print("i end: $i")

        # Move to the next pattern index, or reset to 1 if the end of the pattern is reached
        pattern_index = pattern_index % length(pattern) + 1
    end

    print("\n $splitpoint_adjustments")

    print("\n Length train-data = ", nrow(train), "\n Length eval-data = ", nrow(eval), "\n Length test-data = ", nrow(test))
    
    return train, eval, test

end


split_all_data_advanced_v2 (generic function with 1 method)

In [30]:
#LU
#=summer_training, summer_evaluation, summer_testing = train_eval_test_split(Input_data_summer)

CSV.write("data/$(ID)_summer_train_TOU.csv", summer_training);
CSV.write("data/$(ID)_summer_eval_TOU.csv", summer_evaluation);
CSV.write("data/$(ID)_summer_test_TOU.csv", summer_testing);
=#

In [31]:
#LU
#=# filter winter
Input_data_winter = filter(:winter => !=(0), Input_df)
describe(Input_data_winter), size(Input_data_winter)
=#

In [32]:
#LU
#=winter_training, winter_evaluation, winter_testing = train_eval_test_split(Input_data_winter)

# write data files
CSV.write("data/$(ID)_winter_train_TOU.csv", winter_training);
CSV.write("data/$(ID)_winter_eval_TOU.csv", winter_evaluation);
CSV.write("data/$(ID)_winter_test_TOU.csv", winter_testing);
=#

In [34]:
#all_training, all_evaluation, all_testing = split_all_data_advanced(Input_df)

#LU 
#=
# write data files
CSV.write("data/$(ID)_all_train_TOU.csv", all_training);
CSV.write("data/$(ID)_all_eval_TOU.csv", all_evaluation);
CSV.write("data/$(ID)_all_test_TOU.csv", all_testing);
=#

In [35]:
#LU
#describe(vcat(Input_data_winter, Input_data_summer) ), size(vcat(Input_data_winter, Input_data_summer) )

In [36]:
#LU
#=# both seasons split
both_training, both_evaluation, both_testing = train_eval_test_split(vcat(Input_data_winter, Input_data_summer))

# write data files
CSV.write("data/$(ID)_both_train_TOU.csv", both_training);
CSV.write("data/$(ID)_both_test_TOU.csv", both_testing);
CSV.write("data/$(ID)_both_eval_TOU.csv", both_evaluation);
=#

## Dataset for fixed prices

In [37]:
Input_df[!, "p_buy"] .= 0.3;
Input_df[!, "p_sell"] .= 0.1;

In [56]:
all_training, all_evaluation, all_testing = split_all_data_advanced_v2(Input_df)
for (set, df) in [("train" , all_training), ("eval", all_evaluation), ("test", all_testing)]
    print(set, "-data length: ", nrow(df), "\n")
end

start
 i 1, set_name test, days 10.0, adjustment left: 0
 i: 1, set_name: test, row_count: 240, days: 10.0
 split point not ok at nday 240.
 month 11, day 10. Trying at next day. 
Total new splitpoint adjustment for test: 24
 Splitpoint found.i end: 265
 i 265, set_name eval, days 5.0, adjustment left: 0
 i: 265, set_name: eval, row_count: 120, days: 5.0
 split point not ok at nday 384.
 month 11, day 16. Trying at next day. 
Total new splitpoint adjustment for eval: 24
 Splitpoint found.i end: 409
 i 409, set_name train, days 15.0, adjustment left: 0
 i: 409, set_name: train, row_count: 360, days: 15.0
 split point not ok at nday 768.
 month 12, day 2. Trying at next day. 
Total new splitpoint adjustment for train: 24
 Splitpoint found.i end: 793
 i 793, set_name test, days 9.0, adjustment left: 0
 Splitpoint found.i end: 1009
 i 1009, set_name eval, days 4.0, adjustment left: 0
 Splitpoint found.i end: 1105
 i 1105, set_name train, days 14.0, adjustment left: 0
 Splitpoint found.i en

In [200]:
#LU
#Input_data_summer = filter(:summer => !=(0), Input_df);
#Input_data_winter = filter(:winter => !=(0), Input_df);

In [201]:
#LU summer_training, summer_evaluation, summer_testing = train_eval_test_split(Input_data_summer);
#LU winter_training, winter_evaluation, winter_testing = train_eval_test_split(Input_data_winter);
#LU all_training, all_evaluation, all_testing = train_eval_test_split(Input_df);
#LU both_training, both_evaluation, both_testing = train_eval_test_split(vcat(Input_data_winter, Input_data_summer));

In [202]:
#LU
#=CSV.write("data/$(ID)_both_train_fix.csv", both_training);
CSV.write("data/$(ID)_both_test_fix.csv", both_testing);
CSV.write("data/$(ID)_both_eval_fix.csv", both_evaluation);=#

CSV.write("data/$(ID)_all_train_fix.csv", all_training);
CSV.write("data/$(ID)_all_test_fix.csv", all_testing);
CSV.write("data/$(ID)_all_eval_fix.csv", all_evaluation);

#LU
#=CSV.write("data/$(ID)_summer_train_fix.csv", summer_training);
CSV.write("data/$(ID)_summer_test_fix.csv", summer_testing);
CSV.write("data/$(ID)_summer_eval_fix.csv", summer_evaluation);
CSV.write("data/$(ID)_winter_train_fix.csv", winter_training);
CSV.write("data/$(ID)_winter_test_fix.csv", winter_testing);
CSV.write("data/$(ID)_winter_eval_fix.csv", winter_evaluation);
=#

"data/Charger06_all_eval_fix.csv"